# < 버트 사전학습 모형 가져오기 >
- [허깅페이스와 버트 학습하기](https://github.com/kimwoonggon/publicservant_AI/blob/master/1_(HuggingFace)%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%ED%8F%89%EA%B0%80_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb) 참고 사이트

- [참고 블로그](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=winddori2002&logNo=222022178447)

In [3]:
import tensorflow as tf

import numpy as np
import pandas as pd

# from transformers import *
import transformers

import json

from tqdm import tqdm
import os

In [4]:
os.listdir('bert')

['bert_model.ckpt.data-00000-of-00001',
 '.ipynb_checkpoints',
 'bert_model.ckpt.meta',
 'multi_cased_L-12_H-768_A-12.zip',
 'bert_model.ckpt.index',
 'vocab.txt',
 'bert_config.json']

### 기본 설정

In [23]:
SEQ_LEN = 128
BATCH_SIZE = 20

# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "comment"

# target = 칼럼
LABEL_COLUMN = "label"

# df 호출

In [5]:
df = pd.read_csv('data/train.csv', encoding = 'UTF-8-SIG')
# test_df = pd.read_csv('data/test.csv', encoding = 'UTF-8-SIG')

df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


### 확인용

In [7]:
df.shape

(8367, 4)

In [8]:
print("bias classes: ", df.bias.unique())
print("hate classes: ", df.hate.unique())

bias classes:  ['none' 'others' 'gender']
hate classes:  ['none' 'hate']


In [9]:
# 가능한 모든 조합들의 개수 확인
pd.crosstab(df.bias, df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


# 두 라벨의 가능한 모든 조합 만들기

In [10]:
combinations = np.array(np.meshgrid(df.bias.unique(), df.hate.unique())).T.reshape(-1,2)

combinations

array([['none', 'none'],
       ['none', 'hate'],
       ['others', 'none'],
       ['others', 'hate'],
       ['gender', 'none'],
       ['gender', 'hate']], dtype=object)

## bias, hate 컬럼을 합친 리스트 만들기

In [11]:
bias_hate = list(np.array([df['bias'].values, df['hate'].values]).T.reshape(-1,2))

bias_hate[:5]

[array(['none', 'none'], dtype=object),
 array(['none', 'hate'], dtype=object),
 array(['others', 'hate'], dtype=object),
 array(['none', 'none'], dtype=object),
 array(['none', 'none'], dtype=object)]

## 정수 코드로 라벨 만들기

In [13]:
labels = []

for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

df['label'] = labels
df.head()

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0


# bert input 만들기

- 한글 데이터를 분석하려면, 100개가 넘는 언어에 대해 훈련된 버트를 사용해야 함
- multilingual BERT를 사용할 예정.
- 
모델을 로드하기에 앞서, 토크나이저를 불러오기 : huggingface로 쉽게 토크나이저를 불러오기 가능

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpncd5hue0


storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f


https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpw1_nvn7t


storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29


https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpotrgwucu


storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
creating metadata file for /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingfac

https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmptw8qlsvy


storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

### 가장 기초에 속하는 tokenizer 사용 방법 
#### tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌
#### tokenizer.tokenize => 문장을 토큰화

In [17]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[101, 9356, 11018, 31605, 31605, 110589, 71568, 118913, 11018, 9576, 119281, 9786, 79940, 23811, 40364, 9520, 23160, 102]


In [18]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어', '##맞', '##는', '예', '##측', '카', '##리스', '##마', '없는', '악', '##역']


## bert input = token, segment, position
1. 토큰 : 문장을 토크나이징 한 후, 인덱스 번호를 매긴 것, 단어를 단어사전의 위치값으로 표현
2. 세그먼트 : 예를 들어 문장이 두 개가 있다면, 앞의 문장과 뒤의 문장을 구분하는 것. 파인튜닝 과정에서 앞뒷문장 구분 안할거면 전부 0으로 지정
3. 마스크 : 문장이 유효한 값인지, 아니면 유효하지 않은 값이라 패딩 값으로 채운 것인지를 나타냄 (문장이 유효한 값이면 1, 유효하지 않은 값이면 0으로 채움)
-> 토큰, 세그먼트, 마스크를 인풋으로 버트 모형에 넣으면 버트 모형에 맞게 고차원으로 임베딩이 됨

In [20]:
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=64, pad_to_max_length=True))
# 문장마다 문장 길이는 다르지만, 버트의 인풋 길이는 일정해야 하므로, 버트에서 지정한 문장 길이를 초과하면 패딩값인 0을 채우게 됨

[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
# 마스크 인풋 : 토큰 인풋에서 패딩이 아닌 부분은 1, 패딩인 부분은 0
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (128 - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 정리
- 버트 인풋 = 토큰, 세그먼트, 마스크
```
"전율을 일으키는 영화. 다시 보고싶은 영화" 라는 문장을 가지고 예를 들면,

토큰 인풋 : [101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

세그먼트 인풋 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

마스크 인풋 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

문장들을 버트 인풋으로 바꾸면 ```문장 -> 토큰 인풋, 세그먼트 인풋, 마스크 인풋``` 으로 변환됨
### huggingface에서는 [토큰 인풋, 마스크 인풋, 세그먼트 인풋] 순서!

# 버트 모형의 입력에 맞게 변형해주는 함수
함수 내부에 tokenizer.encode 함수가 버트 모형을 토큰화해주고 토큰화 된 단어를 인덱스에 맞게 숫자로 바꿔줌

In [24]:
def convert_data(data_df):
    global tokenizer
    SEQ_LEN = 128  # 버트에 들어갈 인풋의 길이
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], 
                                 max_length=SEQ_LEN, 
                                 truncation=True, 
                                 padding='max_length')
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트. 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets


# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(data_df):
    
    # data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    
    data_x, data_y = convert_data(data_df)
    
    return data_x, data_y

In [25]:
# 데이터셋 나눠주기
train_size = int(0.7 * len(df))

train_df = df[:train_size]
test_df = df[train_size:].reset_index(drop=True)

In [26]:
train_x, train_y = load_data(train_df)  # train 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test_df)

/tmp/ipykernel_116155/340878336.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
/tmp/ipykernel_116155/340878336.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
100% 5856/5856 [00:01<00:00, 3537.26it/s]
100% 2511/2511 [00:00<00:00, 3522.42it/s]


In [27]:
train_x

[array([[   101,   8935,  83616, ...,      0,      0,      0],
        [   101,   8896, 119449, ...,      0,      0,      0],
        [   101,   9032,  17196, ...,      0,      0,      0],
        ...,
        [   101,  21266,  90947, ...,      0,      0,      0],
        [   101,   9764,  53639, ...,      0,      0,      0],
        [   101,   9670,  14523, ...,      0,      0,      0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

- 사전학습된 버트 모델의 인풋은 문장 토큰화가 숫자로 바뀐 것과, 앞문장인지 뒷문장인지 알려주는 문장 순서 벡터가 들어갑니다. 우리는 문장 하나를 가지고만 훈련할 것이므로 순서 벡터는 모두 0으로 통일합니다.

- 그리고 파인튜닝 시에는 문장 안에 일부 단어를 가리는 마스킹은 사용하지 않습니다.
  - Fine Tuning 이란? - 기존에 학습되어져 있는 모델을 기반으로 아키텍쳐를 새로운 목적(나의 이미지 데이터에 맞게)변형하고 이미 학습된 모델 Weights로 부터 학습을 업데이트하는 방법을 말한다.

In [171]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    segs = []
    
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        print('segments : ', segments)
        indices.append(ids)
        segs.append(segments)
        
    indices = np.array(indices)
    segs = np.array(segs)
    return [indices, segs]


def sentence_load_data(sentences):  # sentence는 List로 받음
    data_x = sentence_convert_data(sentences)
    return data_x

In [172]:
sentence_load_data(["케라스로 버트 해보기 정말 재밌음. 근데 어렵다", "케라스 쉬워? 쉬워!"])

100% 2/2 [00:00<00:00, 702.33it/s]

	 ['[CLS]', '케', '##라', '##스로', '버', '##트', '해', '##보', '##기', '정', '##말', '재', '##밌', '##음', '.', '근', '##데', '어', '##렵', '##다', '[SEP]']
segments :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	 ['[CLS]', '케', '##라', '##스', '쉬', '##워', '?', '쉬', '##워', '!', '[SEP]']
segments :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[array([[   101,   9806,  17342,  94980,   9336,  15184,   9960,  30005,
          12310,   9670,  89523,   9659,    100,  32158,    119,   8926,
          28911,   9546, 118879,  11903,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,   

In [173]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

### < 모델 구조 중 MLM, NSP > 
MLM과 NSP를 통해 token-level, sentence-level로 학습하게 됨  
#### 1. MSM (Masked Language Model)
:  Deep bidirectional representation을 학습하기 위해 input 토큰의 일부분을 랜덤하게 마스킹하는 기법
- mask 토큰의 마지막 hidden vector는 softmax로 들어가 어떤 단어를 출력하는 방식
- mask 토큰은 랜덤하게 만들어지며 그 비율은 전체의 15% : masking 비율의 80%는 mask 토큰으로 변환하며 10%는 랜덤한 단어로 변환하고 마지막 10%는 기존 단어를 사용
- MLM에서는  문장 전체를 예측하는 것이 아닌 masking된 단어만 예측하며 token-level 학습 방법  

=>  MLM을 통한 pre-training된 BERT는 문맥 정보를 활용할 수 있는 모델. mask 토큰은 pre-training에만 사용되며 fine-tuning에서는 사용되지 않음.

#### 2. NSP (Next Sentence Prediction)
- NLP의 Question Answering (QA) and Natural Language Inference(NLI)와 같은 task에서 두 문장 간의 relationship을 파악하는 것이 매우 중요
-  BERT는 두 문장이 연결되어 있던 문장인지 예측하는 next sentence prediction을 학습하여 문장 간의 relationship을 파악할 수 있도록 pre-training
-  학습을 위해  50%는 연결된 문장, 50%는 랜덤하게 뽑힌 문장으로 학습


In [174]:
# 모델 구조 확인  :  총 12층의 트랜스포머 계층이 있음
model.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  91812096    ['Input-Token[0][0]']            
 g)                              (119547, 768)]                                                   
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']      

# 사전학습 모델을 변형
사전학습 파일을 로드하여, 우리가 불러들였던 사전학습 모델을 변형하기. (가장 중요한 부분)
1. input
- input으로는 우리가 문장을 토큰화 하여 숫자로 변형시켜주었던 토큰 벡터와, 앞문장인지 뒷문장인지 알려주는 세그멘트 두 가지 필요 -> ```inputs = modle.inputs[:2]```로 정의
  - model.inputs : Token, Segment, Masked 3가지
2. output
- output은 일단 사전학습 모델을 약간 잘라줌.
- (outputs=Dense(1)) 맨 위 3층을 잘라 냄(NSP-Dense 부분) -> 잘라낸 부분에 label을 알려주는 Dense(6)을 사전학습 모델에 애드온 시켜 주기
  - 참고) Dense(1)은 아웃풋이 하나(1)로, 문장이 긍정에 가까우면 0에 가까운 값을, 부정에 가까우면 1에 가까운 값을 출력해주는 레이어.

## 사전 설치하였던 Radam을 활용
-> deep learning의 기울기 강하 훈련을 하도록 정해주기.
그 다음에 bert_model을 return

In [184]:
def get_bert_finetuning_model(model):
    inputs = model.inputs[:2]  # token, segment, mask 중 mask 제외한 나머지
    dense = model.layers[-3].output  # NSP-Dense 부분

    outputs = keras.layers.Dense(6, 
                                 activation='softmax',
                                 kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                                 name = 'real_output')(dense)

    bert_model = keras.models.Model(inputs, outputs)
    bert_model.compile(
        optimizer=RAdam(learning_rate=1e-6, weight_decay=0.0025),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
  
    return bert_model

In [182]:
model.layers[-3].output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'NSP-Dense')>

## 모델 flow 시각화

In [189]:
from IPython.display import SVG
from keras.utils import model_to_dot

SVG(tf.keras.utils.model_to_dot(get_bert_finetuning_model(model), dpi=65).create(prog='dot', format='svg'))

ImportError: cannot import name 'model_to_dot' from 'keras.utils' (/opt/conda/lib/python3.8/site-packages/keras/utils/__init__.py)

In [190]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline
SVG(model_to_dot(get_bert_finetuning_model(model), dpi=65).create(prog='dot', format='svg'))

ValueError: `tf.compat.v1.keras` Optimizer (<keras_radam.optimizers.RAdam object at 0x7fd12c7c83a0>) is not supported when eager execution is enabled. Use a `tf.keras` Optimizer instead, or disable eager execution.

In [191]:
sess = K.get_session()

uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

AttributeError: module 'tensorflow' has no attribute 'report_uninitialized_variables'

In [192]:
tf.__version__

'2.8.0'

In [193]:
bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

ValueError: `tf.compat.v1.keras` Optimizer (<keras_radam.optimizers.RAdam object at 0x7fd3b1b241c0>) is not supported when eager execution is enabled. Use a `tf.keras` Optimizer instead, or disable eager execution.